In [3]:
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

rangos = pd.DataFrame(index=range(2), columns=range(1))

def CDG(df, a, b, c, d):

    # Diccionario de DF por cada estaciÃ³n
    Estaciones = {}
    percent = {}
    # Percentilies deseados
    Prob = [a, b, c]
    for columna in df.columns:
        # Crear un DataFrame para cada estaciÃ³n y ordenarlo
        est = df[[columna]].dropna().sort_values(by=columna, ascending=False).reset_index(drop=True)
        est = est.iloc[:-3,:]
        est = est.iloc[1:,:]
        est['Probabilidad'] = 100*(est.index + 1) / (len(est) + 1)
        Estaciones[columna] = est
            
        i = 0
        P = pd.DataFrame(np.zeros((len(Prob), 2)), columns=['Nivel', 'Probabilidad'])

        for p in Prob:
            idx = (est.iloc[:, 1] - p).abs().idxmin()
            P.at[i, 'Nivel'] = float("{:.2f}".format(est.iloc[idx, 0]))
            P.at[i, 'Probabilidad'] = float("{:.2f}".format(est.iloc[idx, 1]))
            i = i + 1
        percent[columna] = P.copy()
    rangos.iloc[d,0] = str((a, b, c))
    Percentiles.append(percent)
    
    return Percentiles, rangos  

def LeerArchivos(a):
    # Crear una ventana raíz
    root = tk.Tk()
    root.withdraw()  # Ocultar la ventana principal
    # Hacer que la ventana esté siempre en el frente
    root.attributes('-topmost', True)
    # Abrir un cuadro de diálogo para seleccionar un archivo CSV
    file_path = filedialog.askopenfilename(
        title=f"Selecciona un archivo CSV de {a}",
        filetypes=(("CSV files", "*.csv"), ("Todos los archivos", "*.*")))
    # Mostrar la ruta del archivo seleccionado
    print(f"Archivo seleccionado: {file_path}")

    # Cerrar la ventana raíz
    root.destroy()
        
    if file_path:  # Verificar si se seleccionó un archivo
        try:
            df = pd.read_csv(file_path, index_col=None)
        except Exception as e:
            print(f"Error al leer el archivo: {e}")
    else:
        print("No se seleccionó ningún archivo.")
        
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:,0])
    df = df.set_index(df.columns[0])
        
    return df

def ElegirEstacion(Data, a):
    
    def Seleccion():
        columna_seleccionada = columna_var.get()
        print(f"Has seleccionado la estación: {columna_seleccionada}")
        root.destroy()  # Cierra la ventana
    
    # Crear la ventana principal de Tkinter
    root = tk.Tk()
    root.title("Selecciona una estación de {a}")
    # Hacer que la ventana esté siempre en el frente
    root.attributes('-topmost', True)
    # Variable para almacenar la selección
    columna_var = tk.StringVar()
    columna_var.set(Data.columns[0])  # Valor inicial en el OptionMenu
    dropdown = ttk.OptionMenu(root, columna_var, *Data.columns)
    dropdown.pack(pady=10)
    # Botón para confirmar la selección
    boton = tk.Button(root, text="Seleccionar", command = Seleccion)
    boton.pack(pady=20)
    # Iniciar la ventana
    root.mainloop()
    columna_seleccionada = columna_var.get()
    
    return columna_seleccionada

def resample(N,P):
    a = int(input("Ingrese el número de resampleo: "))
    DatosNiv = N.groupby(N.reset_index().index // a).mean().reset_index(drop=True)
    DatosPrec = P.groupby(P.reset_index().index // a).sum().reset_index(drop=True)
    
    return DatosNiv, DatosPrec, a
    

def ANN():
    # Escalar los datos entre 0 y 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(pd.DataFrame(DatosPrec[EstP]))
    
    # Dividir los datos en conjunto de entrenamiento y prueba (80% - 20%)
    train_size = int(len(data_scaled) * 0.5)
    train, test = data_scaled[:train_size], data_scaled[train_size:]
    
    # Crear una función para transformar los datos en formato adecuado para LSTM (multivariables)
    def create_dataset(data, look_back=10, forecast_horizon=dias_futuros-1):
        X, y = [], []
        for i in range(len(data) - look_back - forecast_horizon):
            X.append(data[i:(i + look_back), :])  # Tomar todas las características
            y.append(data[(i + look_back):(i + look_back + forecast_horizon), 0])  # Predecir la primera variable
        return np.array(X), np.array(y)
    
    # Usamos 90 días para predecir los próximos 7 días
    look_back = 10
    forecast_horizon = dias_futuros-1
    X_train, y_train = create_dataset(train, look_back, forecast_horizon)
    X_test, y_test = create_dataset(test, look_back, forecast_horizon)
    
    # Reshape para que el modelo LSTM lo entienda (samples, time steps, features)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))
    
    # Parámetros de entrenamiento
    epochs = 20
    batch_size = 5
    
    # Crear el modelo LSTM
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))  # Hay 'num_vars' variables de entrada
    model.add(LSTM(50))
    model.add(Dense(forecast_horizon))  # Predecir los próximos 6 días
    
    # Compilar el modelo
    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    # Entrenar el modelo
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=2)
    
    # Realizar el pronóstico
    test_predict = model.predict(X_test)
    
    # Invertir el escalado para las predicciones y los valores reales
    test_predict_unscaled = []
    y_test_unscaled = []
    
    for i in range(test_predict.shape[0]):
        temp_predict = scaler.inverse_transform(np.concatenate([test_predict[i].reshape(-1, 1), np.tile(X_test[i, -1, 1:], (forecast_horizon, 1))], axis=1))[:, 0]
        temp_y_test = scaler.inverse_transform(np.concatenate([y_test[i].reshape(-1, 1), np.tile(X_test[i, -1, 1:], (forecast_horizon, 1))], axis=1))[:, 0]
        test_predict_unscaled.append(temp_predict)
        y_test_unscaled.append(temp_y_test)
    
    test_predict_unscaled = np.array(test_predict_unscaled)
    y_test_unscaled = np.array(y_test_unscaled)

    return test_predict_unscaled

def LogicaDifusa(P, N):
    
    # Antecedentes (precipitaciónn y nivel) y consecuente (Nivel o caudal)
    PrecP = ctrl.Antecedent([P[EstP].min(), Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0], P[EstP].max()], 'Precipitación Pasada')
    PrecP2 = ctrl.Antecedent([P[EstP].min(), Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0], P[EstP].max()], 'Precipitación Pasada2')
    NivP = ctrl.Antecedent([N[EstN].min(), Percentiles[0][EstN].iloc[2,0], Percentiles[0][EstN].iloc[1,0], Percentiles[0][EstN].iloc[0, 0], N[EstN].max()], 'Nivel Pasado')
    Niv = ctrl.Consequent([N[EstN].min(), Percentiles[0][EstN].iloc[2,0], Percentiles[0][EstN].iloc[1,0], Percentiles[0][EstN].iloc[0, 0], N[EstN].max()], 'Nivel')
    
    # Definición de membresí­as
    PrecP['Bajo'] = fuzz.trimf(PrecP.universe, [P[EstP].min(), P[EstP].min(), Percentiles[1][EstP].iloc[2, 0]])
    PrecP['Medio-Bajo'] = fuzz.trimf(PrecP.universe, [P[EstP].min(), Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0]])
    PrecP['Medio'] = fuzz.trimf(PrecP.universe, [Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0]])
    PrecP['Alto'] = fuzz.trapmf(PrecP.universe, [Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0], P[EstP].max(), P[EstP].max()])

    PrecP2['Bajo'] = fuzz.trimf(PrecP.universe, [P[EstP].min(), P[EstP].min(), Percentiles[1][EstP].iloc[2, 0]])
    PrecP2['Medio-Bajo'] = fuzz.trimf(PrecP.universe, [P[EstP].min(), Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0]])
    PrecP2['Medio'] = fuzz.trimf(PrecP.universe, [Percentiles[1][EstP].iloc[2, 0], Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0]])
    PrecP2['Alto'] = fuzz.trapmf(PrecP.universe, [Percentiles[1][EstP].iloc[1, 0], Percentiles[1][EstP].iloc[0, 0], P[EstP].max(), P[EstP].max()])

    NivP['Bajo'] = fuzz.trimf(Niv.universe, [N[EstN].min(), N[EstN].min(), Percentiles[0][EstN].iloc[2, 0]])
    NivP['Medio-Bajo'] = fuzz.trimf(Niv.universe, [N[EstN].min(), Percentiles[0][EstN].iloc[2, 0], Percentiles[0][EstN].iloc[1, 0]])
    NivP['Medio'] = fuzz.trimf(Niv.universe, [Percentiles[0][EstN].iloc[2, 0], Percentiles[0][EstN].iloc[1, 0], Percentiles[0][EstN].iloc[0, 0]])
    NivP['Alto'] = fuzz.trapmf(Niv.universe, [Percentiles[0][EstN].iloc[1, 0], Percentiles[0][EstN].iloc[0, 0], N[EstN].max(), N[EstN].max()])
    
    Niv['Bajo (90-100%)'] = fuzz.trimf(Niv.universe, [N[EstN].min(), N[EstN].min(), Percentiles[0][EstN].iloc[2, 0]])
    Niv['Medio-Bajo (50-90%)'] = fuzz.trimf(Niv.universe, [N[EstN].min(), Percentiles[0][EstN].iloc[2, 0], Percentiles[0][EstN].iloc[1, 0]])
    Niv['Medio (10-50%)'] = fuzz.trimf(Niv.universe, [Percentiles[0][EstN].iloc[2, 0], Percentiles[0][EstN].iloc[1, 0], Percentiles[0][EstN].iloc[0, 0]])
    Niv['Alto (0-10%)'] = fuzz.trapmf(Niv.universe, [Percentiles[0][EstN].iloc[1, 0], Percentiles[0][EstN].iloc[0, 0], N[EstN].max(), N[EstN].max()])

    # REGLAS
    B = ctrl.Rule(PrecP['Bajo'], Niv['Bajo (90-100%)'])
    MB = ctrl.Rule(PrecP['Medio-Bajo'], Niv['Medio-Bajo (50-90%)'])
    M = ctrl.Rule(PrecP['Medio'], Niv['Medio (10-50%)'])
    A = ctrl.Rule(PrecP['Alto'], Niv['Alto (0-10%)'])

    B2 = ctrl.Rule(PrecP2['Bajo'], Niv['Bajo (90-100%)'])
    MB2 = ctrl.Rule(PrecP2['Medio-Bajo'], Niv['Medio-Bajo (50-90%)'])
    M2 = ctrl.Rule(PrecP2['Medio'], Niv['Medio (10-50%)'])
    A2 = ctrl.Rule(PrecP2['Alto'], Niv['Alto (0-10%)'])

    Bn = ctrl.Rule(NivP['Bajo'], Niv['Bajo (90-100%)'])
    MBn = ctrl.Rule(NivP['Medio-Bajo'], Niv['Medio-Bajo (50-90%)'])
    Mn = ctrl.Rule(NivP['Medio'], Niv['Medio (10-50%)'])
    An = ctrl.Rule(NivP['Alto'], Niv['Alto (0-10%)'])
    
    Pronosticos = pd.DataFrame(np.nan, index=DatosPrec[EstP].iloc[-dias_pasados:].index, columns=[f"Día {i+1}" for i in range(dias_futuros)], dtype=float)
    N_Variables = int(input("¿Con qué variables desea trabajar?:\n1. Precipitación t-1\n2. Nivel t-1\n3. Precipitación t-1, t-2\n4. Precipitación t-1 y nivel t-1\n5. Precipitación t-1, t-2 y nivel t-1\n"))
    
    if N_Variables == 1:
        #Parametros para pdf
        text = "1_Variable_P(t-1)"
        estaciones = f'{EstP}'
        REGLAS = "precipitación t-1"
        # Parámetros para pronóstico
        reglas = [B, MB, M, A]
        def FL(i, dia):
            Nivel.input['Precipitación Pasada'] = Prec_In.iloc[i, dia]
            Nivel.compute()
            Pronosticos.iloc[i, dia] = float(Nivel.output['Nivel'])
            DATOS.iloc[i,0] = Prec_In.iloc[i, dia]
            DATOS.iloc[i,3] = float(Nivel.output['Nivel'])
        
    if N_Variables == 2:
        #Parametros para pdf
        text = "1_Variable_N(t-1)"
        estaciones = f'{EstN}'
        REGLAS = "nivel t-1"
        # Parámetros para pronóstico
        reglas = [Bn, MBn, Mn, An]
        def FL(i, dia):
            Nivel.input['Nivel Pasado'] = Nivel_In.iloc[i, dia]
            Nivel.compute()
            Pronosticos.iloc[i, dia] = float(Nivel.output['Nivel'])
            Nivel_In.iloc[i, dia + 1] = float(Nivel.output['Nivel'])
            DATOS.iloc[i,2] = Nivel_In.iloc[i, dia]
            DATOS.iloc[i,3] = float(Nivel.output['Nivel'])
            
    if N_Variables == 3:
        #Parametros para pdf
        text = "2_Variables_P(t-1,t-2)"
        estaciones = f'{EstP}'
        REGLAS = "precipitación t-1 y t-2"
        # Parámetros para pronóstico
        reglas = [B, MB, M, A, B2, MB2, M2, A2]
        def FL(i, dia):
            Nivel.input['Precipitación Pasada'] = Prec_In.iloc[i, dia]
            Nivel.input['Precipitación Pasada2'] = Prec_In2.iloc[i, dia]
            Nivel.compute()
            Pronosticos.iloc[i, dia] = float(Nivel.output['Nivel'])
            DATOS.iloc[i,0] = Prec_In.iloc[i, dia]
            DATOS.iloc[i,1] = Prec_In2.iloc[i, dia]
            DATOS.iloc[i,3] = float(Nivel.output['Nivel'])   
        
    if N_Variables == 4:
        #Parametros para pdf
        text = "2_Variables_P(t-1)_N(t-1)"
        estaciones = f'{EstP}, {EstN}'
        REGLAS = "precipitación t-1 y nivel t-1"
        # Parámetros para pronóstico
        reglas = [B, MB, M, A, Bn, MBn, Mn, An]
        def FL(i, dia):
            Nivel.input['Precipitación Pasada'] = Prec_In.iloc[i, dia]
            Nivel.input['Nivel Pasado'] = Nivel_In.iloc[i, dia]
            Nivel.compute()
            Pronosticos.iloc[i, dia] = float(Nivel.output['Nivel'])
            Nivel_In.iloc[i, dia + 1] = float(Nivel.output['Nivel'])
            DATOS.iloc[i,0] = Prec_In.iloc[i, dia]
            DATOS.iloc[i,2] = Nivel_In.iloc[i, dia]
            DATOS.iloc[i,3] = float(Nivel.output['Nivel']) 
            
    if N_Variables == 5:
        #Parametros para pdf
        text = "3_Variables_P(t-1,t-2)_C(t-1)"
        estaciones = f'{EstP}, {EstN}'
        REGLAS = "precipitación t-1, t-2 y caudal t-1"
        # Parámetros para pronóstico
        reglas = [B, MB, M, A, B2, MB2, M2, A2, Bn, MBn, Mn, An]
        def FL(i, dia):
            Nivel.input['Precipitación Pasada'] = Prec_In.iloc[i, dia]
            Nivel.input['Precipitación Pasada2'] = Prec_In2.iloc[i, dia]
            Nivel.input['Nivel Pasado'] = Nivel_In.iloc[i, dia]
            Nivel.compute()
            Pronosticos.iloc[i, dia] = float(Nivel.output['Nivel'])
            Nivel_In.iloc[i, dia + 1] = float(Nivel.output['Nivel'])
            DATOS.iloc[i,0] = Prec_In.iloc[i, dia]
            DATOS.iloc[i,1] = Prec_In2.iloc[i, dia]
            DATOS.iloc[i,2] = Nivel_In.iloc[i, dia]
            DATOS.iloc[i,3] = float(Nivel.output['Nivel'])
    
    Nivel_ctrl = ctrl.ControlSystem(reglas)
    Nivel = ctrl.ControlSystemSimulation(Nivel_ctrl)
    
    NR = pd.DataFrame(index=DatosPrec[EstP].iloc[-dias_pasados:].index, columns=range(dias_futuros))
    PR = pd.DataFrame(index=DatosPrec[EstP].iloc[-dias_pasados:].index, columns=range(dias_futuros))

    for i in range(dias_futuros):
        NR.iloc[:dias_pasados-i,i] = N[EstN].iloc[-dias_pasados+i:].values
        PR.iloc[:dias_pasados-i,i] = P[EstP].iloc[-dias_pasados+i:].values

    NR = NR.astype(float)
    PR = PR.astype(float)
    
    Prec_In = pd.DataFrame(index=P[EstP].iloc[-dias_pasados:].index, columns=range(dias_futuros))
    Prec_In2 = pd.DataFrame(index=P[EstP].iloc[-dias_pasados-1:-1].index, columns=range(dias_futuros))
    Nivel_In = pd.DataFrame(index=P[EstP].iloc[-dias_pasados:].index, columns=range(dias_futuros + 1))
    
    # Asignar valores de t-1 y t-2 (reales)
    Prec_In[0] = P[EstP].iloc[-dias_pasados:].values.flatten()  
    Prec_In2[0] = P[EstP].iloc[-dias_pasados-1:-1].values.flatten()
    Prec_In2[1] = P[EstP].iloc[-dias_pasados:].values.flatten()
    Nivel_In[0] = DatosNiv[EstN].iloc[-dias_pasados:].values.flatten() 
    
    # Asignar predicciones a las columnas restantes
    for i in range(dias_pasados):
        Prec_In.iloc[i, 1:] = test_predict_unscaled[-dias_pasados+i, :]
        Prec_In2.iloc[i, 2:] = test_predict_unscaled[-dias_pasados-1+i, 1:]
    
    DATOS = pd.DataFrame(index=P[EstP].iloc[-dias_pasados:].index, columns=['Prec_t-1', 'Prec_t-2', 'Niv_t-1', 'Pronostico'])
    
    for i in range(dias_pasados):
        
        a = dias_pasados - i
        
        for dia in range(dias_futuros):
            
            FL(i, dia)
        
    # Calcular las métricas RMSE, MAE y R²
    metrics = {'RMSE': [], 'MAE': [], 'R²': [], 'NSlog': [], 'PBIAS': [], 'MAPE': [], 'RMSPE': []}

    for day in range(dias_futuros):

        a = dias_pasados - day
        
        rmse = np.sqrt(mean_squared_error(NR.iloc[:a, day], Pronosticos.iloc[:a, day]))
        mae = mean_absolute_error(NR.iloc[:a, day], Pronosticos.iloc[:a, day])
        r2 = r2_score(NR.iloc[:a, day], Pronosticos.iloc[:a, day])
        nr_log = np.log(NR.iloc[:a, day] + 1)
        pred_log = np.log(Pronosticos.iloc[:a, day] + 1)
        nslog = 1 - (np.sum((nr_log - pred_log) ** 2) / np.sum((nr_log - np.mean(nr_log)) ** 2))
        pbias = 100 * (np.sum(NR.iloc[:a, day] - Pronosticos.iloc[:a, day]) / np.sum(NR.iloc[:a, day]))
        mape = 100 * np.mean(np.abs((NR.iloc[:a, day] - Pronosticos.iloc[:a, day]) / NR.iloc[:a, day]))
        rmspe = 100 * np.sqrt(np.mean(((NR.iloc[:a, day] - Pronosticos.iloc[:a, day]) / NR.iloc[:a, day]) ** 2))
        metrics['RMSE'].append(rmse)
        metrics['MAE'].append(mae)
        metrics['R²'].append(r2)
        metrics['NSlog'].append(nslog)
        metrics['PBIAS'].append(pbias)
        metrics['MAPE'].append(mape)
        metrics['RMSPE'].append(rmspe)
        
        print(f"Día {day + 1} - RMSE: {rmse}, MAE: {mae}, R²: {r2}, NSlog: {nslog}, PBIAS: {pbias}, MAPE: {mape}, RMSPE: {rmspe}")
    
    return NR, PR, Pronosticos, PrecP, Niv, metrics, DATOS, estaciones, REGLAS
    
def confianza2():
    # Definir los percentiles del eje X
    percentiles_x = [2.5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 97.5]
    percentiles_labels = ['P2.5', 'P10', 'P20', 'P30', 'P40', 'P50', 'P60', 'P70', 'P80', 'P90', 'P97.5']

    # Obtener percentiles reales
    real = NR.iloc[-len(Pronosticos):,0]
    percentilesR = np.percentile(real, percentiles_x)

    # Crear DataFrames para cada día y calcular percentiles y rangos en bucle
    dias_percentiles = {}
    for dia in range(1, dias_futuros+1):
        # Ordenar los valores pronosticados
        ordenados = Pronosticos.iloc[:, dia - 1].dropna().sort_values(ascending=True).reset_index(drop=True)

        # Crear un DataFrame para cada día
        dia_df = pd.DataFrame({'Nombre': percentiles_labels})

        # Calcular los percentiles
        dia_df['percentiles'] = [np.percentile(ordenados, i) for i in percentiles_x]

        # Función para calcular los rangos C10, C90, C5, C95
        def calcular_rang(ordenados, percentil, percentiles_x):
            # Obtener el percentil anterior
            if percentil == percentiles_x[0]:
                p_anterior = 0
            else:
                p_anterior = np.percentile(ordenados, percentiles_x[percentiles_x.index(percentil) - 1])

            # Filtrar los datos que están en la marca de clase
            p = ordenados[(ordenados <= np.percentile(ordenados, percentil)) & (ordenados > p_anterior)]

            # Calcular los percentiles
            c10 = np.percentile(p, 10) if len(p) > 0 else np.nan  # Verificar longitud para evitar errores
            c90 = np.percentile(p, 90) if len(p) > 0 else np.nan
            c5 = np.percentile(p, 5) if len(p) > 0 else np.nan
            c95 = np.percentile(p, 95) if len(p) > 0 else np.nan
            return c10, c90, c5, c95

        # Calcular los rangos para cada percentil y agregar al DataFrame
        dia_df[['C10', 'C90', 'C5', 'C95']] = [calcular_rang(ordenados, i, percentiles_x) for i in percentiles_x]

        # Guardar el DataFrame del día en el diccionario
        dias_percentiles[f'Dia_{dia}'] = dia_df

    return dias_percentiles, percentilesR

def excel():
    
    # Especifica la ruta y el nombre del archivo
    ruta_archivo = f'Series{Resample}Días_Estación{EstN}.xlsx'

    Series = pd.DataFrame(index=DatosPrec[EstP].iloc[-dias_pasados:].index, columns=['Caudal medio real [m3/s]', 'Precipitación real acumulada [mm]'])
    
    Series.iloc[:,0] = NR.iloc[:,0]
    Series.iloc[:,1] = PR.iloc[:,0]
    
    # Crea un archivo Excel y guarda cada DataFrame en una hoja diferente
    with pd.ExcelWriter(ruta_archivo, engine='openpyxl') as writer:
        Series.to_excel(writer, sheet_name='Series', index=False)
        Pronosticos.to_excel(writer, sheet_name='Pronosticos', index=False)
        DATOS.to_excel(writer, sheet_name='Datos', index=False)
    
    print(f"Archivo Excel guardado en {ruta_archivo}")

In [97]:
Datos_Historicos_Nivel = LeerArchivos("datos historicos de nivel")
DatosNiv = LeerArchivos("periodo seleccionado de nivel")
EstN = ElegirEstacion(DatosNiv, "nivel")
DatosPrec = LeerArchivos("periodo seleccionado de precipitacion")
EstP = ElegirEstacion(DatosPrec, "precipitación")
DatosNiv, DatosPrec, Resample = resample(DatosNiv, DatosPrec)

dias_futuros = int(input("Ingrese el número de días futuros: "))

test_predict_unscaled = ANN()
dias_pasados = len(test_predict_unscaled) - 1

Archivo seleccionado: C:/Users/juanj/Downloads/Estaciones/Estaciones/H5025/Seleccionado/Niveles_H5025_relleno.csv
Archivo seleccionado: C:/Users/juanj/Downloads/Estaciones/Estaciones/H5025/Seleccionado/Niveles_H5025_relleno.csv
Has seleccionado la estación: 5025
Archivo seleccionado: C:/Users/juanj/Downloads/Estaciones/Estaciones/H5025/Seleccionado/Prec_H5025_relleno.csv
Has seleccionado la estación: P39


Ingrese el número de resampleo:  7
Ingrese el número de días futuros:  2


Epoch 1/20
27/27 - 9s - loss: 0.1530 - val_loss: 0.1813 - 9s/epoch - 333ms/step
Epoch 2/20
27/27 - 0s - loss: 0.1483 - val_loss: 0.1661 - 403ms/epoch - 15ms/step
Epoch 3/20
27/27 - 0s - loss: 0.1454 - val_loss: 0.1648 - 366ms/epoch - 14ms/step
Epoch 4/20
27/27 - 0s - loss: 0.1453 - val_loss: 0.1646 - 426ms/epoch - 16ms/step
Epoch 5/20
27/27 - 0s - loss: 0.1439 - val_loss: 0.1641 - 392ms/epoch - 15ms/step
Epoch 6/20
27/27 - 0s - loss: 0.1445 - val_loss: 0.1648 - 356ms/epoch - 13ms/step
Epoch 7/20
27/27 - 0s - loss: 0.1440 - val_loss: 0.1642 - 371ms/epoch - 14ms/step
Epoch 8/20
27/27 - 0s - loss: 0.1429 - val_loss: 0.1653 - 366ms/epoch - 14ms/step
Epoch 9/20
27/27 - 0s - loss: 0.1422 - val_loss: 0.1673 - 355ms/epoch - 13ms/step
Epoch 10/20
27/27 - 0s - loss: 0.1434 - val_loss: 0.1640 - 348ms/epoch - 13ms/step
Epoch 11/20
27/27 - 0s - loss: 0.1443 - val_loss: 0.1649 - 349ms/epoch - 13ms/step
Epoch 12/20
27/27 - 0s - loss: 0.1412 - val_loss: 0.1733 - 355ms/epoch - 13ms/step
Epoch 13/20
27/

In [106]:
Percentiles = []
Percentiles, rangos = CDG(DatosNiv, 10, 50, 90, 0)  
Percentiles, rangos = CDG(DatosPrec, 0, 1, 2, 1)

NR, PR, Pronosticos, PrecP, Niv, metrics, DATOS, estaciones, REGLAS = LogicaDifusa(DatosPrec, DatosNiv)
dias_percentiles, percentilesR = confianza2()

¿Con qué variables desea trabajar?:
1. Precipitación t-1
2. Nivel t-1
3. Precipitación t-1, t-2
4. Precipitación t-1 y nivel t-1
5. Precipitación t-1, t-2 y nivel t-1
 5


Día 1 - RMSE: 0.02365050484848759, MAE: 0.01656758327299536, R²: 0.8792538232380742, NSlog: 0.8860030176489175, PBIAS: -1.5844157714551725, MAPE: 4.724164799818205, RMSPE: 6.404194043981315
Día 2 - RMSE: 0.07634721543402821, MAE: 0.05763991648176452, R²: -0.36206838731715485, NSlog: -0.33167379713459666, PBIAS: -3.849589137651387, MAPE: 17.16065471543078, RMSPE: 22.54796499627992


In [107]:
#Generar excel
#excel()

# Generar nombre del archivo PDF dinámico basado en variables y nombres de las columnas
pdf_filename = f'Series{Resample}Días_Estación{EstN}.pdf'

# Crear el archivo PDF para guardar gráficos y métricas
with PdfPages(pdf_filename) as pdf:
    # Página 1: Información general
    plt.figure(figsize=(10, 6))
    plt.text(0.5, 0.9, 'Variables Utilizadas en el Pronóstico:', ha='center', fontsize=14)
    plt.text(0.5, 0.8, f'{estaciones}', ha='center', fontsize=12)
    plt.text(0.5, 0.7, f'Variable Pronosticada: Caudal en estación {EstN}', ha='center', fontsize=14)
    plt.text(0.5, 0.6, f'Reglas basadas en {REGLAS}', ha='center', fontsize=14)
    plt.text(0.5, 0.5, f'Rangos de: Caudal{rangos.iloc[0,0]}, Precipitación{rangos.iloc[1,0]}', ha='center', fontsize=12)
    plt.text(0.5, 0.4, 'Métricas de Ajuste:', ha='center', fontsize=14)
    for i in range(dias_futuros):
        plt.text(0.5, 0.3 - i*0.05, f"Día {i + 1} - RMSE: {metrics['RMSE'][i]:.4f}, MAE: {metrics['MAE'][i]:.4f}, R²: {metrics['R²'][i]:.4f}, NSlog: {metrics['NSlog'][i]:.4f}, PBIAS: {metrics['PBIAS'][i]:.4f}, MAPE: {metrics['MAPE'][i]:.4f}%, RMSPE: {metrics['RMSPE'][i]:.4f}",
                 ha='center', fontsize=11)
    plt.axis('off')
    pdf.savefig()
    plt.close()
    
    # Gráficos de predicción e intervalos de confianza
    for day in range(dias_futuros):

        a = dias_pasados - day
        Dia = dias_percentiles[f'Dia_{day+1}']

        # Crear subplots: 1 fila y 2 columnas
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Gráfico 1: Pronóstico vs Realidad
        ax1.set_title(f'Pronóstico del Día {day+1}')
        ax1.plot(DatosPrec.index[-len(NR):], NR.iloc[:, 0], label= 'Caudal Real')
        ax1.plot(DatosPrec.index[-len(NR):], Pronosticos.iloc[:, day], label= 'Pronóstico', color='red')
        ax1.set_xlabel('Fecha')
        ax1.set_ylabel('Caudal (m3/s)')
        ax1.legend()
        ax1.grid(True)

        # Gráfico 2: Intervalos de confianza
        ax2.set_title(f'Día {day+1} - Intervalo de Confianza')
        ax2.plot(Dia['Nombre'], percentilesR, label='Caudal real', color='black', linewidth=1.5)

        # Graficar los intervalos de confianza
        ax2.fill_between(Dia['Nombre'], Dia['C10'], Dia['C90'], color='blue', alpha=0.2, label='C10-C90')
        ax2.fill_between(Dia['Nombre'], Dia['C5'], Dia['C95'], color='gray', alpha=0.2, label='C5-C95')

        ax2.set_xlabel('Percentil')
        ax2.set_ylabel('Caudal (m3/s)')
        ax2.legend(loc='upper left')
        ax2.grid(True)

        # Ajustar el espaciado entre los subplots
        plt.tight_layout()

        pdf.savefig()
        plt.close()
        
    # Gráfico de MAE, RMSE, R²
    days = range(1, dias_futuros + 1)
    fig, ax1 = plt.subplots(figsize=(10, 5))
    # Gráfico de RMSE y MAE en el eje primario
    ax1.plot(days, metrics['RMSE'], label='RMSE', color='b')
    ax1.plot(days, metrics['MAE'], label='MAE', color='g')
    ax1.set_xlabel('Día de Pronóstico')
    ax1.set_ylabel('RMSE / MAE', color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid(True)
    # Crear el segundo eje (eje secundario) para R²
    ax2 = ax1.twinx()
    ax2.plot(days, metrics['R²'], label='R²', color='r')
    ax2.set_ylabel('R²', color='black')
    ax2.tick_params(axis='y', labelcolor='black')
    # Añadir leyendas para cada eje
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    # Título de la gráfica
    plt.title('Métricas de Pronóstico para Cada Día')
    pdf.savefig()
    plt.close()

    # Gráfico de NSlog, RMSPE, MAPE
    days = range(1, dias_futuros + 1)
    fig, ax1 = plt.subplots(figsize=(10, 5))
    # Gráfico de RMSPE y MAPE en el eje primario
    ax1.plot(days, metrics['RMSPE'], label='RMPSE', color='b')
    ax1.plot(days, metrics['MAPE'], label='MAPE', color='g')
    ax1.set_xlabel('Día de Pronóstico')
    ax1.set_ylabel('RMSPE / MAPE', color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid(True)
    # Crear el segundo eje (eje secundario) para R²
    ax2 = ax1.twinx()
    ax2.plot(days, metrics['NSlog'], label='NSlog', color='r')
    ax2.set_ylabel('NSlog', color='black')
    ax2.tick_params(axis='y', labelcolor='black')
    # Añadir leyendas para cada eje
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    # Título de la gráfica
    plt.title('Métricas de Pronóstico para Cada Día')
    pdf.savefig()
    plt.close()

    # Gráfico de PBIAS
    plt.figure(figsize=(10, 5))
    days = range(1, dias_futuros + 1)
    plt.plot(days, metrics['PBIAS'], label='PBIAS')
    plt.title('Métricas de Pronóstico para Cada Día')
    plt.xlabel('Día de Pronóstico')
    plt.ylabel('Métrica')
    plt.legend()
    plt.grid(True)
    pdf.savefig()
    plt.close()
    
    # Gráfico membresías de precipitación y nivel
    plt.figure(figsize=(10, 5))
    Niv.view()
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(10, 5))
    PrecP.view()
    pdf.savefig()
    plt.close()
    
# Mensaje de finalización
print(f"Resultados y gráficos guardados en el archivo {pdf_filename}.")

Resultados y gráficos guardados en el archivo Series7Días_Estación5025.pdf.


C:\Users\juanj\.conda\envs\Scikit-fuzzi\lib\site-packages\matplotlib\figure.py:456: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  warnings.warn(
C:\Users\juanj\.conda\envs\Scikit-fuzzi\lib\site-packages\matplotlib\figure.py:456: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  warnings.warn(


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>